In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd

path_base_original_real="C:\dataset\original_dataset\\real"
path_base_original_fake="C:\dataset\original_dataset\\fake"
fullpath_fake=path_base_original_fake+"\\fake.csv"
path_base_modified="C:\dataset\processed_datatset"
fullpath_modified=path_base_modified+"\\total_dataset.csv"
path_modified_onlyreal=path_base_modified+"\\real_dataset.csv"
path_modified_onlyfake=path_base_modified+"\\fake_dataset.csv"

In [2]:
from nltk.tokenize import word_tokenize
import re 
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from gensim.models import word2vec
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
# nltk.download('wordnet')

In [4]:
readpath='C:\dataset\\processed_datatset\\total_dataset.csv'
writepath='C:\dataset\\processed_datatset\\small_dataset.csv'
df = pd.read_csv(readpath)
dfr = df[df.label==1]
dff = df[df.label==0]
print(dfr.shape)
print(dff.shape)

dfr=dfr.sample(n=5000)
dff=dff.sample(n=5000)
df=pd.concat([dfr,dff])
print(df.head())
print(df.shape)

# df.rename(columns={"content": "text"},inplace=True)
# print(df.head())

# df["label"].replace({0: "FAKE",1: "REAL"}, inplace=True)
# df


(142570, 3)
(12273, 3)
                                                    title  \
95125   The Taliban think McDonald’s is ‘tasteless and...   
84128              The real reason Trump canned Jim Comey   
122993  Commentary: Why the battle for Aleppo could de...   
2993    I Used an Assault Rifle in the Army. I Don’t T...   
102617  Harry Potter actor Rupert Grint loses £1m tax ...   

                                                  content  label  
95125   McDonald’s has a new    —   the Taliban. The  ...      1  
84128   The straw that broke the camel’s back for   FB...      1  
122993  If Wednesday’s ceasefire in Aleppo holds, it c...      1  
2993    I was 19 the first time I held an assault rifl...      1  
102617  Harry Potter star Rupert Grint has lost his le...      1  
(10000, 3)


In [5]:
df.isnull().sum().sum()

0

In [6]:
df.sample(frac=1)

,title,content,label
19043,Attorney for Illegal Alien in Prom-Night Drunk...,"HOUSTON, Texas — The attorney for an previo...",1
129030,This 3-minute cartoon explains why nuclear wea...,Nuclear weapons are something so scary and so...,1
108125,"New Year, New Laws: States Diverge On Gun Righ...",With the New Year comes a long list of new law...,1
87335,Unmask the foreign buyers stashing cash in Ame...,New York’s swankiest skyscrapers have become t...,1
149338,"Number Of Accusers Grows To 12, As Former Miss...","By Sarah Jones on Thu, Oct 27th, 2016 at 1:41 ...",0
...,...,...,...
154624,Good Samaritan wearing Indian headdress disarm...,Good Samaritan wearing Indian headdress disarm...,0
144952,Apocalyptic sounds in the skies are back! This...,Apocalyptic sounds in the skies are back! This...,0
134814,Here’s how much you would need to afford rent ...,There is nowhere in this country where so...,1
86989,ISIS is killing dozens of its own in hunt for ...,"BAGHDAD — In March, a senior commander with...",1


In [7]:
y=df.label
df=df.drop('label',axis=1)


In [8]:
df

,title,content
95125,The Taliban think McDonald’s is ‘tasteless and...,McDonald’s has a new — the Taliban. The ...
84128,The real reason Trump canned Jim Comey,The straw that broke the camel’s back for FB...
122993,Commentary: Why the battle for Aleppo could de...,"If Wednesday’s ceasefire in Aleppo holds, it c..."
2993,I Used an Assault Rifle in the Army. I Don’t T...,I was 19 the first time I held an assault rifl...
102617,Harry Potter actor Rupert Grint loses £1m tax ...,Harry Potter star Rupert Grint has lost his le...
...,...,...
149066,"Tomgram: Nomi Prins, Too Big to Fail, Hillary-...",(30 fans) - Advertisement - \nThis article ori...
154454,The Truth U.S. Government Does NOT Want You To...,Share on Facebook The people of the world shou...
151737,Media Covering Up MASSIVE Anti-Trump Violence ...,Don’t let the media tell you that the anti-Don...
154061,"Cómo decir «No» a las armas nucleares, por Man...",«El arte de la guerra»\nCómo decir «No» a las ...


In [9]:
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.naive_bayes import GaussianNB

from sklearn import metrics


In [10]:
xtrain,xtest,ytrain,ytest= train_test_split(df['content'],y,test_size=0.33,random_state=53)

In [11]:
countvectorizer =CountVectorizer(stop_words='english' )
ctrainx=countvectorizer.fit_transform(xtrain)
ctestx=countvectorizer.transform(xtest)
print(ctrainx.shape)
print(ctestx.shape)

(6700, 97895)
(3300, 97895)


In [12]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7 )
ttrainx = tfidf_vectorizer.fit_transform(xtrain)
ttestx = tfidf_vectorizer.transform(xtest)
print(ttrainx.shape)
print(ttestx.shape)

(6700, 97895)
(3300, 97895)


In [13]:
# MultinomialNB    count vector
classifier =MultinomialNB(alpha=0.03)
classifier.fit(ctrainx,ytrain)
ctestres=classifier.predict(ctestx)
score =metrics.accuracy_score(ytest,ctestres)
score

0.8451515151515151

In [14]:
# MultinomialNB    tfidf vector
classifier =MultinomialNB(alpha=0.03)
classifier.fit(ttrainx,ytrain)
ttestres=classifier.predict(ttestx)
score =metrics.accuracy_score(ytest,ttestres)
score


0.8572727272727273

In [15]:
clf = svm.SVC(kernel='poly' ,degree=2 ,C=1 ,gamma=0.1)  #kernel  =poly
clf.fit(ttrainx, ytrain)
y_pred = clf.predict(ttestx)
score=metrics.accuracy_score(ytest,y_pred)
score

0.48757575757575755

In [16]:
clf = svm.SVC(kernel='sigmoid')   #sigmoid 
clf.fit(ttrainx, ytrain)
y_pred = clf.predict(ttestx)
score=metrics.accuracy_score(ytest,y_pred)
score

D:\SETUP_FILES_2\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.48757575757575755

In [17]:
clf = svm.SVC(kernel='rbf' ,C=1 ,gamma=0.1)   # kernel='rbf'
clf.fit(ttrainx, ytrain)
y_pred = clf.predict(ttestx)
score=metrics.accuracy_score(ytest,y_pred)
score

0.8703030303030304

In [18]:
clf = svm.SVC(kernel='linear')
clf.fit(ttrainx, ytrain)
y_pred = clf.predict(ttestx)
score=metrics.accuracy_score(ytest,y_pred)
score

0.8942424242424243

In [19]:
import seaborn as sns
color = sns.color_palette()
import matplotlib.pyplot as plt
import lightgbm as lgb # to install $ pip install lightgbm
from PIL import Image

In [ ]:
# dk= 
# sns.pairplot(data=dk, hue="fake", dropna='true')

In [20]:
# !pip install wordcloud
import requests
from wordcloud import WordCloud, STOPWORDS  #install $ pip install wordcloud

# Thanks : https://www.kaggle.com/aashita/word-clouds-of-various-shapes ##
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, figure_size=(24.0,16.0), 
                   title = None, title_size=40, image_color=False):
    stopwords = set(STOPWORDS)
    more_stopwords = {'one', 'br', 'Po', 'th', 'sayi', 'fo', 'Unknown'}
    stopwords = stopwords.union(more_stopwords)

    wordcloud = WordCloud(background_color='white',
                    stopwords = stopwords,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=800, 
                    height=400,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask);
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear");
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud);
        plt.title(title, fontdict={'size': title_size, 'color': 'green', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off');
    plt.tight_layout()  


In [29]:
import numpy as np
response = requests.get('https://raw.githubusercontent.com/manojknit/Natural_Language_Processing/master/images/upvote.png')
from io import BytesIO

In [33]:
# upvote_mask = np.array(Image.open(BytesIO(response.content)))#https://raw.githubusercontent.com/manojknit/Natural_Language_Processing/master/images/upvote.png
# plot_wordcloud(  dff["content"], upvote_mask, max_words=300000, max_font_size=300, title="Word Cloud of Questions")

In [34]:
classifier.coef_

array([[ -9.99302094,  -7.41727091, -14.06365263, ..., -14.06365263,
        -14.06365263, -14.06365263]])

In [35]:
classifier.coef_.shape

(1, 97895)

In [36]:
features = tfidf_vectorizer.get_feature_names()
zip(classifier.coef_ , features)

In [49]:
b= sorted( zip(classifier.coef_[0] , features), reverse=True )
b[:20]     #real

[(-5.550644298395463, 'trump'),
 (-5.640723486283248, 'said'),
 (-6.418407697095881, 'people'),
 (-6.4652964057480355, 'president'),
 (-6.505165660697899, 'clinton'),
 (-6.53205053322892, 'new'),
 (-6.753108022714204, 'like'),
 (-6.768116637540558, 'percent'),
 (-6.800904571330794, 'obama'),
 (-6.811628974861243, 'mr'),
 (-6.851682957251885, 'says'),
 (-6.857946203179813, 'just'),
 (-6.860317837306381, 'state'),
 (-6.8882357705277855, 'house'),
 (-6.930094132218986, 'police'),
 (-6.960833393383645, 'time'),
 (-6.963681068539085, 'told'),
 (-6.989734534059217, 'campaign'),
 (-7.004198185652083, 'republican'),
 (-7.016276569389406, 'year')]

In [51]:
b= sorted( zip(classifier.coef_[0] , features))
b[:20]  #fake

[(-14.063652633567031, '0000'),
 (-14.063652633567031, '0001'),
 (-14.063652633567031, '000202'),
 (-14.063652633567031, '000205'),
 (-14.063652633567031, '000emails'),
 (-14.063652633567031, '001219074822'),
 (-14.063652633567031, '0023z'),
 (-14.063652633567031, '0054z'),
 (-14.063652633567031, '005s'),
 (-14.063652633567031, '00684'),
 (-14.063652633567031, '006s'),
 (-14.063652633567031, '009'),
 (-14.063652633567031, '0099'),
 (-14.063652633567031, '0101'),
 (-14.063652633567031, '015'),
 (-14.063652633567031, '019'),
 (-14.063652633567031, '01am'),
 (-14.063652633567031, '01pm'),
 (-14.063652633567031, '01s'),
 (-14.063652633567031, '020')]

In [52]:
hash_vectorizer = HashingVectorizer(stop_words='english', alternate_sign=False)
hash_train = hash_vectorizer.fit_transform(xtrain)
hash_test = hash_vectorizer.transform(xtest)

In [54]:
clf = MultinomialNB(alpha=.01)
clf.fit(hash_train, ytrain)
pred = clf.predict(hash_test)
score = metrics.accuracy_score(ytest, pred)
score

0.8563636363636363

In [55]:
from sklearn.linear_model import PassiveAggressiveClassifier
clf = PassiveAggressiveClassifier(50)
clf.fit(hash_train, ytrain)
pred = clf.predict(hash_test)
score = metrics.accuracy_score(ytest, pred)

In [56]:
score

0.8854545454545455